In [17]:
from maxim.maxim import Maxim, Config, LoggerConfig
from maxim.logger.components.session import SessionConfig
from maxim.logger.components.trace import TraceConfig
from maxim.logger.components.session import SessionConfig
from maxim.logger.components.span import Span, SpanConfig
from maxim.logger.components.generation import GenerationConfig, GenerationError
import tiktoken
from time import time
import openai
import re
import os
from openai import OpenAI
from uuid import uuid4
from dotenv import load_dotenv




In [ ]:


# setting up maxim logger
maxim = Maxim(Config(apiKey=os.environ["MAXIM_API_KEY"]))
logger =maxim.logger(LoggerConfig(id=os.environ["MAXIM_LOG_REPO"]))
client = OpenAI(api_key=os.environ.get("OpenAI_API_KEY"))


In [7]:
class Agent:
    def __init__(self, client: OpenAI, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        self.generation = None
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, span:Span, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        self.generationConfig = GenerationConfig(id=str(uuid4()), name="generation", provider="OpenAI", model="gpt-4o", model_parameters={"temperature": 0}, messages=self.messages)
        self.generation = span.generation(self.generationConfig)
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
    
        enc = tiktoken.get_encoding("o200k_base")
        llm_output = result
        messages = self.messages
        messages_string = ''.join(["role: " + entry["role"] + " content: " + entry['content'] for entry in messages])
        prompt_tokens = len(enc.encode(messages_string))
        completion_tokens = len(enc.encode(llm_output))

        print(prompt_tokens, completion_tokens)
        
        self.generation.result({
            "id": self.generation.id,
            "object": "text_completion",
            "created": int(time()),
            "model": self.generationConfig.model,
            "choices": [
                {
                    "index": 0,
                    "text": llm_output,
                    "logprobs": None,
                    "finish_reason": "stop",
                },
            ],
            "usage": {
                "prompt_tokens": prompt_tokens,
                "completion_tokens": completion_tokens,
                "total_tokens": prompt_tokens + completion_tokens,
            },
        })

        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o", messages=self.messages
        )
        return completion.choices[0].message.content


In [12]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE 

You will be called again with this:

Observation: France is a country. The capital is Paris.
Thought: I think I have found the answer
Action: Paris.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on Wikipedia
Action: wikipedia : mass of earth
PAUSE

You will be called again with this: 

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()


In [13]:
import re
import httpx
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]
#
def calculate(operation: str) -> float:
    return eval(operation)

In [14]:
session = logger.session(SessionConfig(id=uuid4().hex))
trace = session.trace(TraceConfig(id=uuid4().hex))

def run(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "wikipedia"]

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        span = trace.span(SpanConfig(id=str(uuid4()), name=f"Span : {i}"))
        result = agent(span,next_prompt)
        # print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            print(action)
            span.name = f"Span : {i} - {action}"
            
            chosen_tool = action[0][0]
            print(chosen_tool,"hello")
            tool_span = span.span(SpanConfig(id=str(uuid4()), name=f"Tool Call {chosen_tool}"))
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                tool_span.event(str(uuid4()), f"Tool Call - choosen_tool:args: {chosen_tool}:{arg}",{})
                tool_span.event(str(uuid4()), f"Tool Call - result: {result_tool}",{})
                next_prompt = f"Observation: {result_tool}"
                # print(choosen_tool,"hello")
                # tool_span.end()

            else:
                next_prompt = "Observation: Tool not found"
                span.event(str(uuid4()), f"Tool not found",{})
            print(next_prompt)
            continue

        if "Answer" in result:
            span.event(str(uuid4()), f"Final Answer: {result}",{})   
            break
        span.end()
        
    trace.end()
        

In [18]:
run(query="How many planets in the solar system have rings? Now, calculate the total area of all the rings in the solar system.")


460 37
[('wikipedia', 'rings of planets in the solar system')]
wikipedia hello
Observation: Kirkwood gaps Outer <span class="searchmatch">Solar</span> <span class="searchmatch">System</span> Outer <span class="searchmatch">planets</span> Jupiter ♃ Moons <span class="searchmatch">of</span> Jupiter Io Europa Ganymede Callisto <span class="searchmatch">Rings</span> <span class="searchmatch">of</span> Jupiter Magnetosphere <span class="searchmatch">of</span> Jupiter Jupiter trojans
619 43
[('wikipedia', 'planets with rings in the solar system')]
wikipedia hello
Observation: asteroids Kirkwood gaps Outer <span class="searchmatch">Solar</span> <span class="searchmatch">System</span> Outer <span class="searchmatch">planets</span> Jupiter ♃ Moons of Jupiter Io Europa Ganymede Callisto <span class="searchmatch">Rings</span> of Jupiter Magnetosphere of Jupiter
783 79
[('wikipedia', 'rings of Jupiter')]
wikipedia hello
Observation: The <span class="searchmatch">rings</span> <span class="searchmat